In [1]:
from pyspark.sql import SparkSession, SQLContext 
from pyspark.sql.functions import lower
from operator import add
import re
from collections import OrderedDict
from operator import itemgetter 
import itertools
import json

import pyspark.sql.functions as f  

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("blameyben_lecture1_simple_example")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()

spark_context = spark_session.sparkContext #added
sqlContext = SQLContext(spark_context)



In [2]:
from pyspark.sql.types import *
schema = StructType([])



In [3]:
import timeit
start_time = timeit.default_timer()

#file = open('testfiles/sample_data.json')
file = open('testfiles/sample_data-Copy1.json')

content = file.read()
array = content.split('\n')

file.close()
    
df = sqlContext.createDataFrame(spark_context.emptyRDD(), schema)

for comment in array:
    
    rddjson = spark_context.parallelize([comment])  
    current_df = sqlContext.read.json(rddjson)     
    columns_to_drop = ['author_cakeday','id', 'id_copy']
    if 'author_cakeday' in current_df.columns:
        current_df = current_df.drop('author_cakeday')  
    if 'permalink' in current_df.columns:
        current_df = current_df.drop('permalink')  
    current_df = current_df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','retrieved_on','stickied','subreddit_id')
  
    count = df.count()
    
    if count == 0:
        df = current_df
    else:
        df = df.union(current_df)


elapsed = timeit.default_timer() - start_time

#df.show()

print("Time elapsed: ",elapsed) 
    

Time elapsed:  346.49747060937807


In [4]:
start_time = timeit.default_timer()

#df.select("author").show()
df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed = timeit.default_timer() - start_time

print("Time elapsed: ",elapsed)

+-------------+-----+
|    subreddit|count|
+-------------+-----+
|    AskReddit|    5|
|          CFB|    5|
|   CrazyIdeas|    3|
|      Android|    2|
|relationships|    2|
|       gaming|    2|
|       hockey|    2|
|    2007scape|    2|
|   The_Donald|    2|
|  hearthstone|    2|
+-------------+-----+
only showing top 10 rows

Time elapsed:  19.048326823860407


In [5]:
start_time = timeit.default_timer()

df = df.withColumn('wordCount', f.size(f.split(f.col('body'), ' ')))
df.show()

df.groupBy('wordCount').count().sort("count", ascending = False).show(10)   



print("Time elapsed: ",elapsed)

+------------------+--------------------+----------------+-----------+------+-----+------------------+---------+
|            author|                body|controversiality|created_utc|gilded|score|         subreddit|wordCount|
+------------------+--------------------+----------------+-----------+------+-----+------------------+---------+
|          Dethcola|            A quarry|               0| 1506816000|     0|    3|          sandiego|        2|
|        PennyBotV2|[Salutations! I'm...|               0| 1506816000|     0|    3|              RWBY|        9|
|       Sir_Firebum|I got into baseba...|               0| 1506816000|     0|    2|          baseball|       27|
|         deanzynut|        FUCKING TORY|               0| 1506816000|     0|   18|         2007scape|        2|
|   OfullOstomacheO|I see a water dra...|               0| 1506816000|     0|    1| mildlyinteresting|        6|
|           PlusOn3|Wait. The Michiga...|               0| 1506816000|     0|    1|            C

In [6]:
start_time = timeit.default_timer()

df_stats = df.select(f.mean(f.col('wordCount')).alias('mean')).collect()  #

mean = df_stats[0]['mean']

print("Average wordcount in comment: ", mean, " words.")

print("Time elapsed: ",elapsed)

Average wordcount in comment:  29.71  words.
Time elapsed:  19.048326823860407


+-----------------+-----+
|extracted_country|count|
+-----------------+-----+
|                 |  100|
+-----------------+-----+

Time elapsed:  19.048326823860407
